# Convert XGBoost Pipeline to ONNX

First, serialize your scikit-learn + XGBoost pipeline into ONNX, which serves as the foundation for applying quantization transforms.

In [1]:
#!pip install onnxmltools

In [2]:
import pandas as pd
import numpy as np
import joblib
from onnxmltools import convert_xgboost
from onnxmltools.convert.common.data_types import FloatTensorType
from sklearn.preprocessing import OneHotEncoder

In [3]:
# Reload your data just to infer dimensions
df = pd.read_csv('/content/drive/MyDrive/Fraud Detection/bank_transactions_featured.csv')

In [4]:
df


,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,...,is_large_transaction,log_transaction_amount,transaction_hour,transaction_day_of_week,odd_hour_transaction,user_transaction_count,user_avg_transaction_amount,deviation_from_user_avg,user_primary_location,is_unusual_location
0,TX000001,AC00128,14.09,2023-04-11 16:29:00,Debit,San Diego,D000380,162.198.218.92,M015,ATM,...,0,2.714032,16,1,0,6,304.550000,-290.460000,Chicago,1
1,TX000002,AC00455,376.24,2023-06-27 16:44:00,Debit,Houston,D000051,13.149.61.4,M052,ATM,...,0,5.932882,16,1,0,7,304.622857,71.617143,Baltimore,1
2,TX000003,AC00019,126.29,2023-07-10 18:16:00,Debit,Mesa,D000235,215.97.143.157,M009,Online,...,0,4.846468,18,0,0,4,237.047500,-110.757500,Louisville,1
3,TX000004,AC00070,184.50,2023-05-05 16:32:00,Debit,Raleigh,D000187,200.13.225.150,M002,Online,...,0,5.223055,16,4,0,7,249.954286,-65.454286,Charlotte,1
4,TX000005,AC00411,13.45,2023-10-16 17:51:00,Credit,Atlanta,D000308,65.164.3.100,M091,Online,...,0,2.670694,17,0,0,6,280.796667,-267.346667,Atlanta,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2367,TX002508,AC00297,856.21,2023-04-26 17:09:00,Credit,Colorado Springs,D000625,21.157.41.17,M072,Branch,...,1,6.753683,17,2,0,10,273.373000,582.837000,Chicago,1
2368,TX002509,AC00322,251.54,2023-03-22 17:36:00,Debit,Tucson,D000410,49.174.157.140,M029,Branch,...,0,5.531570,17,2,0,9,340.741111,-89.201111,Albuquerque,1
2369,TX002510,AC00095,28.63,2023-08-21 17:08:00,Debit,San Diego,D000095,58.1.27.124,M087,Branch,...,0,3.388787,17,0,0,7,249.978571,-221.348571,Atlanta,1
2370,TX002511,AC00118,185.97,2023-02-24 16:24:00,Debit,Denver,D000634,21.190.11.223,M041,Online,...,0,5.230948,16,4,0,4,312.650000,-126.680000,Charlotte,1


In [5]:
# load your encoder
ohe = joblib.load('/content/drive/MyDrive/Fraud Detection/model_training/onehot_encoder.pkl')

In [6]:
ohe

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [7]:
# pick numeric cols exactly as in your training script:
numeric_cols = [
    'TransactionAmount','CustomerAge','TransactionDuration','LoginAttempts',
    'AccountBalance','is_large_transaction','log_transaction_amount',
    'transaction_hour','transaction_day_of_week','odd_hour_transaction',
    'user_transaction_count','user_avg_transaction_amount','deviation_from_user_avg'
]

In [8]:
# build one-row array
row = df.iloc[[0]]
X_num = row[numeric_cols].to_numpy()
X_cat = ohe.transform(row[['TransactionType','Location','Channel','CustomerOccupation','user_primary_location']])
feature_dim = X_num.shape[1] + X_cat.shape[1]

In [9]:
# Load your trained XGBoost model
xgb = joblib.load('/content/drive/MyDrive/Fraud Detection/model_training/xgb_model.pkl')

In [10]:
xgb

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [11]:
# Rename feature names inside the booster
booster = xgb.get_booster()
n_features = len(booster.feature_names)
booster.feature_names = [f"f{i}" for i in range(n_features)]
# Update the model’s booster reference
xgb._Booster = booster

In [12]:
# Define ONNX input shape
feature_dim = n_features  # total numeric + one-hot dims
initial_type = [('float_input', FloatTensorType([None, feature_dim]))]

In [13]:
# Convert to ONNX
onnx_model = convert_xgboost(xgb, initial_types=initial_type)
with open('/content/drive/MyDrive/Fraud Detection/model_training/fraud_detector.onnx', 'wb') as f:
    f.write(onnx_model.SerializeToString())

#Ternary Quantization

Once you have fraud_detector.onnx:

Merge initializers and apply 1-bit quantization via QONNX:

In [26]:
!qonnx-convert /content/drive/MyDrive/Fraud Detection/model_training/fraud_detector.onnx

qonnx-convert: Received extra arguments: Detection/model_training/fraud_detector.onnx
Usage: qonnx-convert [OPTIONS] input-model-file


In [27]:
!qonnx-convert "/content/drive/MyDrive/Fraud Detection/model_training/fraud_detector.onnx"

qonnx-convert: Missing required arguments: --output-style
Usage: qonnx-convert [OPTIONS] input-model-file


In [28]:
import os
print(os.listdir("/content/drive/MyDrive/Fraud Detection/model_training"))

['isolation_forest.pkl', 'onehot_encoder.pkl', 'xgb_model.pkl', 'classification_report.txt', 'roc_curve.png', 'fraud_detector.onnx']


In [29]:
import onnxruntime as ort
sess = ort.InferenceSession(
    "/content/drive/MyDrive/Fraud Detection/model_training/fraud_detector.onnx",
    providers=["CPUExecutionProvider"]
)
print("Quantized ONNX loaded successfully!")

Quantized ONNX loaded successfully!


In [30]:
!cp "/content/drive/MyDrive/Fraud Detection/model_training/fraud_detector.onnx" \
    "/content/drive/MyDrive/Fraud Detection/model_training/fraud_detector_1bit.onnx"
!qonnx-convert "/content/drive/MyDrive/Fraud Detection/model_training/fraud_detector.onnx"

qonnx-convert: Missing required arguments: --output-style
Usage: qonnx-convert [OPTIONS] input-model-file


Now fraud_detector.onnx is ternary, and fraud_detector_1bit.onnx holds your original model.

**Why This Works**

Single‐step CLI: Avoids Python import issues (no ONNXMergeInitializer import needed)
.
Ternary Quantization: Maps 1bit weights to {−1, 0, +1}, matching BitNet’s 1.58-bit format
.
QCDQ‐style Output: Inserts QuantizeLinear→Clip→DequantizeLinear nodes around your weight tensors, enabling standard ONNX Runtime execution